In [1]:
# Import libraries and utils
%run "utils_for_doubao.ipynb"

Name: openai
Version: 1.43.1
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: F:\anaconda\Lib\site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [ ]:
os.environ['ARK_API_KEY'] = ''
# gets API Key from environment variable OPENAI_API_KEY
# Get api key
load_dotenv()
OPENAI_API_KEY = os.getenv('ARK_API_KEY')
client = OpenAI(
    api_key=os.environ.get("ARK_API_KEY"),
    base_url="https://ark.cn-beijing.volces.com/api/v3",
)

In [3]:
sampled_avg_df = pd.read_csv('f9_samples.csv')
sampled_avg_df

,Unnamed: 0.1,Unnamed: 0,ID,word1,word2,PoS,Annotator 1,Annotator 2,Annotator 3,Annotator 4,Annotator 5,Annotator 6,Annotator 7,Annotator 8,Annotator 9,Annotator 10,Annotator 11,avg_anno
0,11,1276,1277,开心,高兴,adjectives,6,6,5,5,5,5,6,6,6,5,6,5.55
1,54,1285,1286,孩子气,傻,adjectives,3,1,5,3,4,5,2,3,2,2,2,2.91
2,59,1172,1173,生病,疯,adjectives,0,3,0,1,0,6,0,0,0,0,0,0.91
3,18,1082,1083,残忍,不公平,adjectives,0,0,0,1,0,2,0,0,0,0,2,0.45
4,26,1174,1175,怪诞,奇怪,adjectives,4,6,5,5,4,6,4,3,5,3,6,4.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,408,1590,1591,做,赚,verbs,0,0,0,2,0,2,0,0,0,0,0,0.36
114,429,1357,1358,赦免,原谅,verbs,0,5,4,6,5,5,2,5,4,4,2,3.82
115,438,1324,1325,敬礼,演出,verbs,0,0,0,0,0,0,0,0,0,0,0,0.00
116,448,1363,1364,拖着脚走,骑,verbs,0,0,0,0,0,0,0,0,0,0,0,0.00


In [4]:
# Convert to tuple
tuples_list = list(zip(sampled_avg_df['word1'], sampled_avg_df['word2']))

In [5]:
prompt = ("Rate the semantic similarity of the word pair: [('{word1}'), ('{word2}')] on a scale from 0 to 10, "
          "where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. "
          "Use two decimals. The response should strictly adhere to the structure: [('word1', 'word2', <score>)]. "
          "Do not provide additional explanations or context.")
prompt_cn = ("请给该组词语的语义相似性按照从0到6的刻度打分：[('{word1}'), ('{word2}')]，"
             "其中0代表语义完全不相似，6代表语义完全一致。"
             "所给分数仅保留整数。回答应该严格按照如下格式："
             "[('词语1'，'词语2'，<分数>)]。"
             "请勿提供任何多余的解释或上下文。")

In [6]:
# Define model
model = "ep-20241006053554-cz7vx"

# Set sample size
sample_size = 15

# Delay between individual API calls
delay = 15.0

# Define number of sublists
n_sublists = 20

In [7]:
chunks_of_pairs = split_into_n_lists(tuples_list, 118)
#print_prompts_single(chunks_of_pairs, sample_size, prompt_cn)

In [8]:
# Load encoding
encoding = tiktoken.get_encoding("cl100k_base")

# Count tokens per chunk
token_counts = count_tokens_with_tiktoken(chunks_of_pairs, prompt_cn)

# Show results
print("Token counts for each formatted prompt:", token_counts)

Token counts for each formatted prompt: [142, 145, 144, 144, 145, 145, 142, 142, 142, 143, 142, 143, 145, 143, 143, 143, 144, 141, 147, 146, 145, 146, 148, 142, 142, 142, 143, 140, 146, 144, 142, 142, 144, 142, 144, 144, 143, 146, 143, 142, 145, 146, 143, 143, 142, 143, 143, 146, 144, 142, 142, 143, 147, 145, 146, 142, 140, 144, 145, 142, 142, 142, 145, 141, 145, 141, 144, 143, 146, 142, 144, 139, 140, 143, 142, 142, 142, 145, 141, 143, 150, 143, 155, 148, 142, 146, 144, 149, 143, 144, 143, 143, 145, 143, 145, 144, 144, 140, 142, 141, 140, 142, 140, 144, 142, 142, 145, 144, 146, 139, 141, 141, 140, 141, 144, 144, 149, 142]


In [9]:
len(token_counts*15)

1770

In [10]:
# Get results from API
response = get_responses_single(prompt_cn, chunks_of_pairs, model, sample_size, delay)

Processing: 100%|█████████████████████████████████████████████████████████████| 1770/1770 [7:47:19<00:00, 15.84s/chunk]

Total time taken: 28039.15 seconds


In [35]:
# Define filepath
file_path = './doubao/f9.json'

# Check if file already exists
if not os.path.exists(file_path):
    with open(file_path, 'w') as f:
        json.dump(response, f)
        print("File saved successfully.")
else:
    print("File already exists. JSON was not saved to prevent overwriting.")

File saved successfully.


In [37]:
# Process data into dictionary
data_dict = process_responses(response)

# Check for values higher/lower then sample size
higher_lower_samples = {key: value for key, value in data_dict.items() if len(value) < sample_size or len(value) > sample_size}

# Show results
print(higher_lower_samples)

{}


In [39]:
# Print duplicate word pairs
print_duplicate_word_pairs(sampled_avg_df, data_dict)

Empty DataFrame
Columns: [Combined_Columns]
Index: []
Empty DataFrame
Columns: [Combined_Columns]
Index: []


In [41]:
# Convert dict to Pandas DataFrame
df = create_dataframe(data_dict)

# Show results
df

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
0,开心,高兴,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
1,孩子气,傻,0.0,0.0,2.0,2.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,2.0,2.0,1.0,1.0
2,生病,疯,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,残忍,不公平,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,怪诞,奇怪,5.0,5.0,5.0,4.0,4.0,5.0,4.0,5.0,5.0,5.0,4.0,4.0,4.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,做,赚,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,赦免,原谅,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0,4.0,3.0,4.0,3.0,3.0,4.0
115,敬礼,演出,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
116,拖着脚走,骑,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
# Count null values
count_null_values = df.isnull().sum()

# Show results
print("Null value counts per column:", count_null_values)

Null value counts per column: word1                  0
word2                  0
similarity_score_1     0
similarity_score_2     0
similarity_score_3     0
similarity_score_4     0
similarity_score_5     0
similarity_score_6     0
similarity_score_7     0
similarity_score_8     0
similarity_score_9     0
similarity_score_10    0
similarity_score_11    0
similarity_score_12    0
similarity_score_13    0
similarity_score_14    0
similarity_score_15    0
dtype: int64


In [45]:
# Check for rows with at least one null value
rows_with_null = df[df.isnull().any(axis=1)]

# Show results
rows_with_null

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15


In [47]:
# Define file_path
file_path = './doubao/f9.csv'

# Check if file already exists
if not os.path.exists(file_path):
    df.to_csv(file_path, index=False)
    print("File saved successfully.")
else:
    print("File already exists. Dataframe was not saved to prevent overwriting.")

File saved successfully.
